In [39]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [40]:
#DO NOT CHANGE THIS PART!!
#Function definitions

def create_np_arry_from_ASC_file (num_header, file_name):
    """This function creates from an .asc file an numpy array exlcuding the lines of the header 

    Args:
        num_header ([int]): [number of lines from .asc file that are not integrated in the numpy array]
        file_name ([str]): [Path to the .asc file]

    Returns:
        [numpy array]: [An Array with all the elements from the asc file exluding the header ]
    """
    x = num_header
    ### Creating Numpy Array of 
    with open(file_name, 'r') as handle:
        text = handle.readlines()

    processed_txt = []   
    for line in text[num_header:]: #skip the header
        element= line.strip(" ")
        element= element.replace("\n", "")
        element= element.split(" ")
        processed_txt.append(element)

    np_raw = np.array(processed_txt)
    np_raw =np_raw.astype(float)
    return np_raw

def ACC (num_both_snow, num_both_NOT_snow, num_sample):
    acc = (num_both_snow + num_both_NOT_snow) / num_sample
    return acc

def BIAS (num_snow_observed, num_snow_modeled):
    bias = num_snow_modeled/ num_snow_observed
    return bias

def CSI(num_both_snow, num_sample, num_both_NOT_snow):
    csi = (num_both_snow / (num_sample - num_both_NOT_snow))
    return csi

def valided_area_snow_modeld(validation_date, np_observe, np_model, snow_treshold):
    print(f"\nValidating at spatial scale of the  snow_cover from {validation_date} ")
    if np_observe.shape != np_model.shape:
        print(f"the shape of validation array is {np_observe.shape} and of the model array it is {np_model.shape}")
        raise ValueError("Both input arrays need to have the same size of col & rows -> return to the input files!")
    
    observed_snow = 0
    modeld_snow = 0
    both_snow = 0
    both_NOT_snow = 0
    unqueal = 0
    no_value = 0

    for i in range(np_model.shape[0]):
        for j in range(np_model.shape[1]):
            if np.isnan(np_model[i,j]) == True or np_observe[i,j] == 205:
                no_value  += 1
                continue
            elif np_model[i,j] > snow_treshold and np_observe[i,j] == 100: #is snow
                both_snow +=1
            elif np_model[i,j] <= snow_treshold and np_observe[i,j] == 0:
                both_NOT_snow += 1
            else:
                unqueal += 1

            if np_model[i,j] >= snow_treshold:
                modeld_snow += 1
            if np_observe[i,j] == 100:
                observed_snow += 1
    sample = np_model.shape[0]*np_model.shape[1] - no_value
    acc = ACC(both_snow, both_NOT_snow, sample)
    bias = BIAS(observed_snow, modeld_snow)
    csi = CSI(both_snow, sample, both_NOT_snow)
    print("Accuracy: ", acc, "Bias: ", bias, "Critical success Index: ", csi)
    return (acc, bias, csi)
    
def valided_area_snow_modeld_return_asc_file(validation_date, np_observe, np_model, snow_treshold, header):
    print(f"Creating a .asc file showing the validation of at spatial scale of the  snow_cover from {validation_date} ")
    if np_observe.shape != np_model.shape:
        print(f"the shape of validation array is {np_observe.shape} and of the model array it is {np_model.shape}")
        raise ValueError("Both input arrays need to have the same size of col & rows -> return to the input files!")
    
    snow_treshold
    modeld_snow_TRUE = 1
    modeld_NO_snow_TRUE = 0 
    modeld_snow_FALSE = -1
    modeld_NO_snow_FALSE = -5

    validation_array = np.full(np_model.shape, np.nan)

    for i in range(np_model.shape[0]):
        for j in range(np_model.shape[1]):
            if np.isnan(np_model[i,j]) == True or np_observe[i,j] == 205:
                validation_array[i,j] = 999
            elif np_model[i,j] > snow_treshold and np_observe[i,j] == 100: #is snow
                validation_array[i,j] = modeld_snow_TRUE
            elif np_model[i,j] <= snow_treshold and np_observe[i,j] == 0:
                validation_array[i,j] =modeld_NO_snow_TRUE
            elif np_model[i,j] > snow_treshold and np_observe[i,j] == 0:
                validation_array[i,j] = modeld_snow_FALSE
            else:
                validation_array[i,j] = modeld_NO_snow_FALSE
    np.savetxt(f"Diff_Illustrating_{validation_date}.asc", validation_array, header=header, delimiter=" ", fmt="%1.0f")
    with open(f"Diff_Illustrating_{validation_date}.asc", 'r') as handle:
        text = handle.readlines()
        for i in range(len(text)-1):
            if text[i] == "# \n":
                del text[i]
            if text[i][0] == "#":
                text[i] = text[i][2:]
            else:
                text[i] = f" {text[i]}"   
    with open(f"Diff_Illustrating_{validation_date}.asc", 'w') as f:
        for i in text:
            f.write(i)

def validation_area_import_keyfigures_asc_file (a_observe_name, a_model_name, snow_treshold): 
    # The file must be stored in the folder results
    input_file_observ = f"results/{a_observe_name}.asc"
    input_file_model = f"results/{a_model_name}.asc"

    with open(input_file_observ, "r") as header:
        a = header.readlines()[:5]
        head = ""
        head_rofental_asc_files = head.join(a)

    observe1_raw = create_np_arry_from_ASC_file(5, input_file_observ)
    model1_raw = create_np_arry_from_ASC_file(5, input_file_model)

    valided_area_snow_modeld(f"{a_model_name}", observe1_raw, model1_raw, snow_treshold)
    valided_area_snow_modeld_return_asc_file(f"{a_model_name}", observe1_raw, model1_raw, snow_treshold, head_rofental_asc_files)
    
def validation_area_import_keyfigures_asc_file_SEVERAL_models (observed, lst_models, snow_treshold):
    LEN = len(lst_models)
    for i in range(LEN):
        validation_area_import_keyfigures_asc_file(observed, lst_models[i], snow_treshold)
        

def importance_of_snowtreshold_validation_area_import_keyfigures_several_models (a_observe_name, a_model_name, snow_treshold):
     # The file must be stored in the folder results
    input_file_observ = f"results/{a_observe_name}.asc"
    input_file_model = f"results/{a_model_name}.asc"

    with open(input_file_observ, "r") as header:
        a = header.readlines()[:5]
        head = ""
        head_rofental_asc_files = head.join(a)

    observe1_raw = create_np_arry_from_ASC_file(5, input_file_observ)
    model1_raw = create_np_arry_from_ASC_file(5, input_file_model)

    acc_lst = list()
    bias_lst = list()
    cis_lst = list()
    
    for e in range(len(snow_treshold)):
        acc, bias, cis = valided_area_snow_modeld(f"{a_model_name}", observe1_raw, model1_raw, snow_treshold[e])
        acc_lst.append(acc)
        bias_lst.append(bias)
        cis_lst.append(cis)
    keyfigures_with_colors_and_labels = ((acc_lst, "red", "ACC"), (bias_lst, "green", "BIAS"), (cis_lst, "blue", "CIS"))
    plt.figure(figsize=(7, 3), constrained_layout=True)
    for value, color, name in keyfigures_with_colors_and_labels:
        plt.plot(snow_treshold, value, color=color, label=name)
    plt.title(f"Auswirkung der Höhe des Schneehöhenschwellenwertes \nauf die Validierung vom {a_model_name}")
    plt.xlabel("Schwellenwert der Schneehöhe(m)")
    plt.legend()
    plt.savefig(f"{a_model_name}_schwellenwert_vergleich.svg",dpi=350)

# Start here
Dear user below you might start to adapte the code so that it fits your models, and the name of them.
The function defined below **model_lst** creates a list of all the models that you want to validate.
If you stored your .asc files with a name convention like: "DATE" + "MODEL_CHARACTERISTICS" than you might use the function below and change the "MODEL_CHARACTERISTICS" in the example case `_MODEL`, `_MODEL-SCF1,2`, `_MODEL-SCF1,4`

otherwise you might create a list containing the exact name (with out file extension) for each date you want to validat
`MODEL_lst = ["model1", "second_model", "3_model"]`


In [41]:
def model_lst(a_date):
    MODEL_lst = [f"{a_date}_MODEL", f"{a_date}_MODEL-SCF1,2", f"{a_date}_MODEL-SCF1,4"]
    return MODEL_lst

If you do not use the model_lst function, below you need to exchange the second argument `model_lst("something")`with your created list. IF you do not use a list object the function wont work! If you would like to use a diffrent snow-treshold change the respective variable.

In [42]:

schnee_schwellenwert = 0.03
validation_area_import_keyfigures_asc_file_SEVERAL_models("2020-04-11", model_lst("2020-04-11"), schnee_schwellenwert)
print("\n\nNEXT DATE: \n")
validation_area_import_keyfigures_asc_file_SEVERAL_models("2020-06-02", model_lst("2020-06-02"), schnee_schwellenwert)
print("NEXT DATE: \n")
validation_area_import_keyfigures_asc_file_SEVERAL_models("2020-07-05", model_lst("2020-07-05"), schnee_schwellenwert)


Validating at spatial scale of the  snow_cover from 2020-04-11_MODEL 
Accuracy:  0.9235713636776597 Bias:  1.0525590754561958 Critical success Index:  0.9224693679122257
Creating a .asc file showing the validation of at spatial scale of the  snow_cover from 2020-04-11_MODEL 

Validating at spatial scale of the  snow_cover from 2020-04-11_MODEL-SCF1,2 
Accuracy:  0.9334923230671391 Bias:  1.0702706425854953 Critical success Index:  0.9327527684767981
Creating a .asc file showing the validation of at spatial scale of the  snow_cover from 2020-04-11_MODEL-SCF1,2 

Validating at spatial scale of the  snow_cover from 2020-04-11_MODEL-SCF1,4 
Accuracy:  0.9287635141273735 Bias:  1.076297495289215 Critical success Index:  0.9283442691864822
Creating a .asc file showing the validation of at spatial scale of the  snow_cover from 2020-04-11_MODEL-SCF1,4 


NEXT DATE: 

NEXT DATE: 



In [43]:

schnee_lst = [i*0.002 for i in range(31)]
print(model_lst("2020-06-02")[2])
importance_of_snowtreshold_validation_area_import_keyfigures_several_models("2020-06-02",model_lst("2020-07-05")[2], schnee_lst)
importance_of_snowtreshold_validation_area_import_keyfigures_several_models("2020-07-05",model_lst("2020-07-05")[2], schnee_lst)

2020-06-02_MODEL-SCF1,4

Validating at spatial scale of the  snow_cover from 2020-07-05_MODEL-SCF1,4 
Accuracy:  0.6517836640145029 Bias:  1.9904496904769542 Critical success Index:  0.37623762376237624

Validating at spatial scale of the  snow_cover from 2020-07-05_MODEL-SCF1,4 
Accuracy:  0.6503937959512539 Bias:  0.5230217788754531 Critical success Index:  0.372774778290943

Validating at spatial scale of the  snow_cover from 2020-07-05_MODEL-SCF1,4 
Accuracy:  0.6498902205660188 Bias:  0.5216185008179106 Critical success Index:  0.37175780007518144

Validating at spatial scale of the  snow_cover from 2020-07-05_MODEL-SCF1,4 
Accuracy:  0.6498015912982174 Bias:  0.5212175642300414 Critical success Index:  0.3715351581139114

Validating at spatial scale of the  snow_cover from 2020-07-05_MODEL-SCF1,4 
Accuracy:  0.6495397320978951 Bias:  0.5206161593482375 Critical success Index:  0.37104249058295313

Validating at spatial scale of the  snow_cover from 2020-07-05_MODEL-SCF1,4 
Accura